## Calling Prompts with LLMS

In [1]:
import config

config.load_key("OPENAI_API_KEY")

In [2]:
# Prompts and LLMs

from langchain.llms import OpenAI

# temperatue is a value between 0 and 1
# - closer to 0 makes the OpenAI response more deterministic
# - closer to 1 makes the OpenAI response more creative and unpredictable
# default Open AI model is text-davinci-003
llm_oai = OpenAI(temperature=0)
llm_oai_high_temp = OpenAI(temperature=0.9)

prompt = "What would a good company name be for a company that makes colorful socks?"

response = llm_oai_high_temp(prompt)

In [3]:
print(response)



"Rainbow Socks Co." or "Vibrant Feet Co."


In [4]:
result = llm_oai_high_temp.generate([prompt]*5)

In [5]:
for company_name in result.generations:
    print(company_name[0].text)



"Rainbow Socks Co." or "Vibrant Feet Co."


"Rainbow Socks Co." or "Vibrant Feet Co."


"Rainbow Socks Co." or "Vibrant Feet Co."


"Rainbow Socks Co." or "Vibrant Feet Co."


"Rainbow Socks Co." or "Vibrant Feet Co."


## Using Different Large Language Models

In [6]:
config.load_key("HUGGINGFACEHUB_API_TOKEN")

In [7]:
from langchain import HuggingFaceHub

llm_hf = HuggingFaceHub(repo_id="google/flan-t5-base", model_kwargs={"temperature": 0, "max_length": 64})

prompt_2 = "What are good fitness tips?"

response_2 = llm_hf(prompt_2)

/home/vscode/.local/lib/python3.11/site-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'InferenceApi' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


In [8]:
print(response_2)

Exercise for at least 30 minutes a day.


## Prompt Templating and Chaining

In [9]:
# Prompt Template: Reproducible way to generate a prompt
from langchain import PromptTemplate

template = "You are a naming consultant for new companies. What is a good name for a company that makes {product}?"

prompt_3 = PromptTemplate.from_template(template)
print(prompt.format(product="colorful socks"))

What would a good company name be for a company that makes colorful socks?


In [10]:
# LLM Chain = Prompt + LLM
from langchain import LLMChain

chain = LLMChain(llm=llm_oai, prompt=prompt_3)

In [11]:
response_3 = chain.run("colorful socks")

print(response_3)



"Rainbow Threads"


In [12]:
template_2 = "You are a naming consultant for new companies. What is a good name for a {company} that makes {product}?"
prompt_4 = PromptTemplate.from_template(template_2)

chain_2 = LLMChain(llm=llm_oai, prompt=prompt_4)

In [13]:
response_4 = chain_2.run({
    "company":"ABC Startup",
    "product":"colorful socks",
})

print(response_4)



"Rainbow Threads Co."


## Using Simple Sequential Chaines

In [14]:
from langchain.chains import SimpleSequentialChain

template_3 = "Write a catch phrase for the following company: {company_name}"
prompt_5 = PromptTemplate.from_template(template_3)
chain_3 = LLMChain(llm=llm_oai, prompt=prompt_5)

overall_chain = SimpleSequentialChain(chains=[chain, chain_3], verbose=True)

catchphrase = overall_chain.run("colorful socks")



> Entering new SimpleSequentialChain chain...


"Rainbow Threads"


"Color your world with Rainbow Threads!"

> Finished chain.


In [15]:
print(catchphrase)



"Color your world with Rainbow Threads!"


## Action Agents

In [16]:
# Agent interfaces provide a flexible way to chain calls to LLMs and other tools based on user input
# Two types of agents:
# - Action Agents: at each timestep, decide on the next action using the outputs of all previous actions
# - Plan-and-execute agents: decide on the full sequence of actions up front, then execute them all without updating the plan

# Look at the following prompt.
# The AI is very likely to get the second question wrong and somewhat likely to get the first one incorrect
# We can use agent tools to answer them both correctly
prompt_6 = "When was the 3rd president of the United States born? What is the year raised to the power of 3?"

In [17]:
import pprint
from langchain.agents import get_all_tool_names

pp = pprint.PrettyPrinter(indent=4)

# Print all available tools available in LangChain
pp.pprint(get_all_tool_names())

[   'requests',
    'requests_get',
    'requests_post',
    'requests_patch',
    'requests_put',
    'requests_delete',
    'terminal',
    'sleep',
    'wolfram-alpha',
    'google-search',
    'google-search-results-json',
    'searx-search-results-json',
    'bing-search',
    'metaphor-search',
    'ddg-search',
    'google-lens',
    'google-serper',
    'google-scholar',
    'google-finance',
    'google-trends',
    'google-jobs',
    'google-serper-results-json',
    'searchapi',
    'searchapi-results-json',
    'serpapi',
    'dalle-image-generator',
    'twilio',
    'searx-search',
    'merriam-webster',
    'wikipedia',
    'arxiv',
    'golden-query',
    'pubmed',
    'human',
    'awslambda',
    'stackexchange',
    'sceneXplain',
    'graphql',
    'openweathermap-api',
    'dataforseo-api-search',
    'dataforseo-api-search-json',
    'eleven_labs_text2speech',
    'google_cloud_texttospeech',
    'reddit_search',
    'news-api',
    'tmdb-api',
    'podcast-api',


In [18]:
from langchain.agents import load_tools, initialize_agent

tools = load_tools(["wikipedia", "llm-math"], llm=llm_oai)

# Zero-shot ReAct is an agent type that uses the ReAct framework that determines
# which tool to use based solely on the tools description
agent = initialize_agent(tools, llm_oai, agent="zero-shot-react-description", verbose=True)

agent.run(prompt_6)




> Entering new AgentExecutor chain...
 I should use Wikipedia to find information about the 3rd president of the United States and use a calculator to raise a number to a power.
Action: Wikipedia
Action Input: 3rd president of the United States
Observation: Page: Thomas Jefferson
Summary: Thomas Jefferson (April 13, 1743 – July 4, 1826) was an American statesman, diplomat, lawyer, architect, philosopher, and Founding Father who served as the third president of the United States from 1801 to 1809. He was the primary author of the Declaration of Independence. Following the American Revolutionary War and prior to becoming president in 1801, Jefferson was the nation's first U.S. secretary of state under George Washington and then the nation's second vice president under John Adams.
During the American Revolution, Jefferson represented Virginia at the Second Continental Congress and served as the second governor of Virginia from 1779 to 1781. In 1785, Congress appointed Jefferson U.S. min

'5295319407'

## Human as Tool

In [19]:
from langchain.agents import AgentType

In [20]:
tools_2 = load_tools(["human"])

agent_2 = initialize_agent(
    tools_2,
    llm_oai,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

In [21]:
# agent_2.run("What's my friend Andi's surname?")

## Plan and Execute Agents

In [22]:
# First plan the series of steps, and then execute them all in series

prompt_7 = "Where are the next summer olympics going to be hosted? What is the population of that country raised to the 0.43 power?"

# The Agent needs to follow something like these steps:
# 1. Search for location
# 2. Identify the country
# 3. Search for the population of the country
# 4. Raise the population to 0.43

In [23]:
config.load_key("SERPAPI_API_KEY")

In [24]:
from langchain.chat_models import ChatOpenAI
from langchain_experimental.plan_and_execute import PlanAndExecute, load_agent_executor, load_chat_planner
from langchain import SerpAPIWrapper, LLMMathChain, WikipediaAPIWrapper
from langchain.agents.tools import Tool

llm_math_chain = LLMMathChain.from_llm(llm=llm_oai, verbose=True)
search = SerpAPIWrapper()
wikipedia = WikipediaAPIWrapper()

tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events",
    ),
    Tool(
        name="Wikipedia",
        func=wikipedia.run,
        description="useful for when you need to look up facts and statistics",
    ),
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about math",
    ),
]

In [25]:
# planner and executor should use memory i.e. chat models to leverage previous information
model = ChatOpenAI(temperature=0)
planner = load_chat_planner(model)
executor = load_agent_executor(model, tools, verbose=True)
agent = PlanAndExecute(planner=planner, executor=executor, verbose=True)

In [27]:
# agent.run(prompt_7)

## Memory and Chat Bots

In [31]:
from langchain import ConversationChain

conversation = ConversationChain(llm=llm_oai, verbose=True)

print(conversation.predict(input="Hi there!"))
print(conversation.predict(input="Can we talk about the weather?"))
print(conversation.predict(input="It's a beautiful day"))



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi there!
AI:

> Finished chain.
 Hello! It's nice to meet you. I am an AI created by OpenAI. I am constantly learning and improving my abilities through machine learning algorithms. How can I assist you today?


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there!
AI:  Hello! It's nice to meet you. I am an AI created by OpenAI. I am constantly learning and improving my abilities t

In [33]:
# Interactive Loop Example

# print("Welcome to your AI Chatbot! What's on your mind?")
# conversation = ConversationChain(llm=llm_oai, verbose=True)
# for _ in range(0,3):
#     human_input = input("You: ")
#     ai_response = conversation.predict(input=human_input)
#     print(f"AI: {ai_response}")

## Storing and Retrieving Chat History

In [36]:
from langchain.memory import ChatMessageHistory, ConversationBufferMemory
from langchain.schema import messages_from_dict, messages_to_dict

history = ChatMessageHistory()
history.add_user_message("Hello! Let's talk about giraffes.")
history.add_ai_message("Hi! I'm down to talk about giraffes.")

cache = messages_to_dict(history.messages)
print(cache)

[{'type': 'human', 'data': {'content': "Hello! Let's talk about giraffes.", 'additional_kwargs': {}, 'type': 'human', 'example': False}}, {'type': 'ai', 'data': {'content': "Hi! I'm down to talk about giraffes.", 'additional_kwargs': {}, 'type': 'ai', 'example': False}}]


In [38]:
messages = messages_from_dict(cache)

history = ChatMessageHistory(messages=messages)
buffer = ConversationBufferMemory(chat_memory=history)
conversation_2 = ConversationChain(llm=llm_oai, memory=buffer, verbose=True)

In [40]:
print(conversation_2.predict(input="What are they?"))



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hello! Let's talk about giraffes.
AI: Hi! I'm down to talk about giraffes.
Human: What are they?
AI:

> Finished chain.
 Giraffes are tall, long-necked mammals that are native to the African savannah. They are the tallest land animals in the world, with adult males reaching heights of up to 18 feet. They have distinctive spotted coats and long, dark tongues that they use to reach leaves on tall trees.


## Document Loading and Retrieval Chains

In [42]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA

loader = TextLoader("2023-state-of-the-union.txt")
documents = loader.load()
print(documents)

[Document(page_content='The United States Capitol\n\nMr. Speaker. Madam Vice President. Our First Lady and Second Gentleman.\n\nMembers of Congress and the Cabinet. Leaders of our military.\n\nMr. Chief Justice, Associate Justices, and retired Justices of the Supreme Court.\n\nAnd you, my fellow Americans.\n\nI start tonight by congratulating the members of the 118th Congress and the new Speaker of the House, Kevin McCarthy.\n\nMr. Speaker, I look forward to working together.\n\nI also want to congratulate the new leader of the House Democrats and the first Black House Minority Leader in history, Hakeem Jeffries.\n\nCongratulations to the longest serving Senate Leader in history, Mitch McConnell.\n\nAnd congratulations to Chuck Schumer for another term as Senate Majority Leader, this time with an even bigger majority.\n\nAnd I want to give special recognition to someone who I think will be considered the greatest Speaker in the history of this country, Nancy Pelosi.\n\nThe story of Ame

In [46]:
# Splits recursively first on \n\n, then \n, and then " "
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents=documents)

print(texts[0])
print(texts[1])

page_content='The United States Capitol\n\nMr. Speaker. Madam Vice President. Our First Lady and Second Gentleman.\n\nMembers of Congress and the Cabinet. Leaders of our military.\n\nMr. Chief Justice, Associate Justices, and retired Justices of the Supreme Court.\n\nAnd you, my fellow Americans.\n\nI start tonight by congratulating the members of the 118th Congress and the new Speaker of the House, Kevin McCarthy.\n\nMr. Speaker, I look forward to working together.\n\nI also want to congratulate the new leader of the House Democrats and the first Black House Minority Leader in history, Hakeem Jeffries.\n\nCongratulations to the longest serving Senate Leader in history, Mitch McConnell.\n\nAnd congratulations to Chuck Schumer for another term as Senate Majority Leader, this time with an even bigger majority.\n\nAnd I want to give special recognition to someone who I think will be considered the greatest Speaker in the history of this country, Nancy Pelosi.' metadata={'source': '2023-st

In [47]:
# "Relatedness" embeddings: semantic, etc
embeddings = OpenAIEmbeddings()

In [48]:
# Chroma is an Open Source vector database (embedding database)
store = Chroma.from_documents(texts, embeddings, collection_name="state-of-the-union")

In [49]:
chain = RetrievalQA.from_chain_type(llm_oai, retriever=store.as_retriever())

In [50]:
chain.run("What did Biden talk about Ohio?")

' Biden talked about the Brent Spence bridge between Kentucky and Ohio, the Ironworkers Local 44 in Cincinnati, and Intel building semiconductor factories outside of Columbus, Ohio. He also mentioned the economic impact of these projects on the state of Ohio.'